In [1]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import os
import shutil
import zipfile
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import requests

In [2]:


# Define a new input layer that accepts grayscale (1-channel) images
input_layer = Input(shape=(224, 224, 1))

# Use Conv2D to change 1-channel input to 3-channel so it can work with MobileNetV2
x = Conv2D(3, (3, 3), padding='same', activation='relu')(input_layer)

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Modify the first layer to accept 1-channel input and broadcast to 3 channels
input_layer = Input(shape=(224, 224, 1))  # Grayscale input
x = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(input_layer)  # Convert grayscale to RGB

# Connect the base model
base_model_output = base_model(x)

# Add custom layers on top of the base model
# (e.g., Global Average Pooling, Dense for classification, etc.)
x = GlobalAveragePooling2D()(base_model_output)
x = Dense(1, activation='sigmoid')(x)  # For binary classification (person vs non-person)

# Create the final model
model = Model(inputs=input_layer, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [3]:
# model = tf.keras.models.load_model('model/model_geeks.h5')
model.load_weights('model/model_geeks.h5')

In [4]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 2,225,153 (8.49 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [ ]:
# Apply quantization
quantize_model = tfmot.quantization.keras.quantize_model
quantized_model = quantize_model(model)

# Compile the quantized model
quantized_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Save the quantized model
quantized_model.save('quantized_model.keras')


In [7]:
# # Load the quantized model
# quantized_model = tf.keras.models.load_model('quantized_model.keras')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('model_geeks.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\xxnab\AppData\Local\Temp\tmpa46l0qy8\assets


INFO:tensorflow:Assets written to: C:\Users\xxnab\AppData\Local\Temp\tmpa46l0qy8\assets


Saved artifact at 'C:\Users\xxnab\AppData\Local\Temp\tmpa46l0qy8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name='keras_tensor_156')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1586950019152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1586950019344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1586950018000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1586950018768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1586950019920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1586950020304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1586950018384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1586950019728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1586950020688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1586950020880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1